# Introduction to Azure OpenAI Function Calling


## Objective

This notebook demonstrates the following:

1. Generative AI
1. Function calling


## Time

You should expect to spend 2 minutes running this sample.


## About this example

The objective of the provided Python file is to learn about Azure OpenAI Function Calling using the Azure OpenAI API.


## Installation

Refer to the README.md file in this folder for installation instructions.


## Parameters


In [1]:
import os
import json
import sqlite3
import pandas as pd
from openai import AzureOpenAI
from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Any, Callable, Dict

load_dotenv(".env")

api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")

con = sqlite3.connect("./database/contoso-sales.db")

instructions = None

In [2]:
def log_message(message: str):
    display(HTML(f'<span style="color: red;"><strong>{message}</strong></span>'))

In [3]:
def display_wrapped(message: str):
    wrapped_content = f'<pre style="white-space: pre-wrap; word-wrap: break-word;">{message}</pre>'
    display(HTML(wrapped_content))

In [4]:
def get_revenue_by_region(region: str = None) -> pd.DataFrame:
    params = []
    query = """
        SELECT
            region as Region,
            SUM(number_of_orders) AS Orders,
            SUM(revenue) AS  Revenue,
            SUM(discount) AS Discount
        FROM sales_data
    """

    if region:
        query += "WHERE UPPER(Region) = UPPER(?)\n"
        params.append(region)

    query += "GROUP BY Region ORDER BY Region"

    return pd.read_sql_query(query, con, params=params)

In [5]:
def get_sales_by_month(month: int) -> pd.DataFrame:
    query = """
        SELECT   
            SUM(revenue) AS Revenue, 
            year || '-' || CASE 
                WHEN month < 10 THEN '0' || month 
                ELSE month 
                END AS Month
        FROM sales_data 
        WHERE Month = ? 
        GROUP BY Month, Year 
        ORDER BY Year, Month
    """

    return pd.read_sql_query(query, con, params=[month])

### Create an AzureOpenAI client


In [6]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

### Define the Assistant tools


In [7]:
tools_list = [
    {
        "type": "function",
        "function": {
            "name": "get_revenue_by_region",
            "description": "Get the sales revenue for Contoso by region.",
            "parameters": {
                "type": "object",
                "properties": {
                    "region": {"type": "string"},
                },
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_sales_by_month",
            "description": "Get the Contoso sales by month.",
            "parameters": {
                "type": "object",
                "properties": {
                    "month": {
                        "type": "integer",
                        "description": "The Contoso sales for a specific month.",
                    }
                },
                "required": ["month"],
            },
        },
    },
]

### Process Function calling


In [8]:
def call_functions(tool_calls) -> None:
    function_map: Dict[str, Callable[[Any], pd.DataFrame]] = {
        "get_revenue_by_region": lambda args: get_revenue_by_region(args.get("region", None)),
        "get_sales_by_month": lambda args: get_sales_by_month(args["month"]),
    }

    for tool_call in tool_calls:
        func_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        log_message(f"Function Name: {func_name}, Function Args: {arguments}")

        function = function_map.get(func_name)
        if not function:
            raise ValueError(f"Unknown function: {func_name}")

        result_df = function(arguments)
        display(result_df)

## Instructions for role system assignment


In [9]:
instructions = (
    "You are a sales analysis assistant for Contoso. "
    "Please be polite, professional, helpful, and friendly. "
    "You get all the sales data from this app using the functions provided. "
    "If a question is not related to sales or you cannot answer the question, "
    "say, 'contact IT for more assistance.' "
    "If the user asks for help or says 'help', provide a list of sample questions that you can answer."
)

## Construct the Assistant and send to Azure OpenAI


In [10]:
def process_message(question: str):

    messages = [{"role": "system", "content": instructions}, {"role": "user", "content": question}]

    response = client.chat.completions.create(
        model=api_deployment_name,
        messages=messages,
        tools=tools_list,
    )

    response_message = response.choices[0].message
    tool_calls = getattr(response_message, "tool_calls", [])

    if tool_calls:
        call_functions(tool_calls)
    else:
        display_wrapped(response_message.content)

In [11]:
process_message("What is the revenue for the region of Africa?")

,Region,Orders,Revenue,Discount
0,AFRICA,2948,378710.0,26352.1


In [12]:
process_message("What is the revenue for the region of north america?")

,Region,Orders,Revenue,Discount
0,NORTH AMERICA,3220,458770.0,30409.64


In [13]:
process_message("What is the revenue for all regions?")

,Region,Orders,Revenue,Discount
0,AFRICA,2948,378710.0,26352.10
1,ASIA-PACIFIC,3480,485514.0,43108.78
2,EUROPE,3736,599380.0,48349.88
3,LATIN AMERICA,3768,343626.0,27196.82
4,MIDDLE EAST,3814,507112.0,34766.32
5,NORTH AMERICA,3220,458770.0,30409.64


In [14]:
process_message("What were the sales for April?")

,Revenue,Month
0,26854.0,2021-04
1,41956.0,2022-04
2,103554.0,2023-04
3,74410.0,2024-04


In [15]:
process_message("What is the meaning of life?")